In [36]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from joblib import load
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
# import dill
from utils.PreprocessingMethods import NullEncoder, TimeConversation,ObjectToFloat,OneHotEncoding,Scaling
from utils import split_data

In [38]:
X_train,y_train = split_data.get_train_data()

In [39]:
X_test,y_test = split_data.get_test_data()

In [40]:


# Define the model pipeline
model_pipeline = Pipeline([
    ('preprocessing', load('pipelines/PreprocessingPipeline2.0.pkl')),
    ('xgboost', XGBClassifier()),
])

# Fit the model pipeline to the training data
model_pipeline.fit(X_train, y_train)
predict = model_pipeline.predict(X_test)
print(recall_score(predict,y_test))


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:Time: object

In [ ]:
loaded_pipeline = joblib.load('preprocessing_pipeline.joblib')
X_processed_data = loaded_pipeline.transform(X_train)

# group / ensemble of models
estimator = []
estimator.append(('LR', LogisticRegression()))
estimator.append(('SVC', SVC(kernel='linear')))
estimator.append(('DTC', DecisionTreeClassifier()))
estimator.append(('RFC', RandomForestClassifier()))
  
  
# Voting Classifier with hard voting
vot_hard = VotingClassifier(estimators = estimator, voting ='hard')
vot_hard.fit(X_processed_data, y_train)
y_pred = vot_hard.predict(loaded_pipeline.transform(X_test))

In [ ]:
print(recall_score(y_pred,y_test))

In [ ]:
# Load the pipeline from disk using joblib
loaded_pipeline = joblib.load('preprocessing_pipeline.joblib')
X_processed_data = loaded_pipeline.transform(X_train)


# fit model no training data
model_xgb = XGBClassifier()
model_xgb.fit(X_processed_data, y_train)

In [ ]:
#!pip install xgboost

In [ ]:
## Pending!!!!!


# rskf = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 2, random_state = 42)

In [4]:
y_pred_xgb = model_xgb.predict(loaded_pipeline.transform(X_test))

NameError: name 'model_xgb' is not defined

In [5]:
print(recall_score(y_pred_xgb,y_test))

NameError: name 'y_pred_xgb' is not defined

In [6]:
# !pip install import-ipynb


In [7]:

print(processed_data)

NameError: name 'processed_data' is not defined

In [62]:
data = pd.read_csv('../data/raw/data.csv',  encoding= 'unicode_escape')
data = data.drop(["Unusual"], axis=1)
pipeline = Pipeline([
    ('preprocessing', load('pipelines/PreprocessingPipeline2.0.pkl'))
])
#X_transformed = pipeline.fit_transform(data)
#X_transformed
#feature_names = x_data.named_transformers_['encoder'].get_feature_names_out()



# with open('pipelines/PreprocessingPipeline2.0.pkl', 'rb') as f:
#     transformed = dill.load(f)

# data = transformed.fit_transform(data)


# Get the output feature names

#column_transformer = ColumnTransformer([
#    ('preprocessing_pipeline', preprocessing_pipeline)
#])

#feature_names = column_transformer.named_transformers['cat'] \
#                                .named_steps['encoder'] \
#                                .get_feature_names_out()
# Fit and transform the data using the pipeline
#transformed_data = pipeline.fit_transform(data)
#pd.DataFrame(pipeline.fit_transform(data)).to_csv(test.csv, index=False)

AttributeError: Can't get attribute 'NullEncoder' on <module '__main__'>